In [141]:
#Import necessary packages
import pandas as pd
from bs4 import BeautifulSoup
import requests as re

import time
import os
import nltk
from nltk.util import ngrams
from collections import Counter

In [142]:
#Read in data
data = pd.read_csv('starting_file.csv')
data.head()

,job_title,org_name,salary_min,salary_max,pay_type,peer_specialist_flag,require_certification_before_hire,require_certification_after_hire,college_required,url,do_not_overwrite_flag
0,Peer Recovery Specialist,Mile High Behavioral Healthcare,17.29,21.29,Hourly,1,NaN,NaN,NaN,https://www.simplyhired.com/job/FMWclnhxTd_xau...,NaN
1,Peer Support Specialist,ROCKY MOUNTAIN HUMAN SERVICES,20,22,Hourly,1,1.0,NaN,NaN,https://www.linkedin.com/jobs/view/peer-suppor...,NaN
2,Peer Specialist,WellPower,25,25,Hourly,1,NaN,NaN,NaN,https://www.linkedin.com/jobs/view/peer-specia...,NaN
3,Peer Specialist Case Manager,Jefferson Center for Mental Health,"39,100","49,800",Annual,1,NaN,NaN,1.0,https://www.linkedin.com/jobs/view/peer-specia...,NaN
4,Peer Support Specialist,University of Colorado,NaN,NaN,NaN,1,NaN,1.0,NaN,https://www.linkedin.com/jobs/view/peer-suppor...,NaN


In [143]:
#Evaluate duplicate titles/org names
#Not currently used, but should use later
with open(f'./jd-data/dups.txt','w') as f:    
    f.write(str(data[['job_title','org_name','url']].groupby(['job_title','org_name']).count()))

In [144]:
#Pull html files from data file
for i in range(0,data.shape[0]):
    num = i
    filename = f'{data.org_name[num]} - {data.job_title[num]}'

    if not os.path.exists(f'./jd-data/{filename}.txt') and data.do_not_overwrite_flag[i] != 1:
        r = re.get(data.url[num])
        with open(f'./jd-data/{filename}.txt','w') as f:
            f.write(r.text)
        time.sleep(3)

In [145]:
#Extract text from html files
for i in range(0,data.shape[0]):
    num = i
    filename = f'{data.org_name[num]} - {data.job_title[num]}'
    if os.path.exists(f'./jd-data/{filename}.txt'):
        with open(f'./jd-data/{filename}.txt','r') as f:
            soup = BeautifulSoup(f, 'html.parser')
        with open(f'./jd-soup/soup - {filename}.txt','w') as f:
            f.write(soup.get_text().strip())


In [146]:
#Normalize data in job postings and find bigrams

token_list = []
bigram_list = []

for i in range(0,data.shape[0]):
    num = i
    filename = f'{data.org_name[num]} - {data.job_title[num]}'
    with open(f'./jd-soup/soup - {filename}.txt','r') as f:
        test = f.read()

    tokens = nltk.word_tokenize(test.replace('\n',' ').strip())
    from nltk.corpus import stopwords
    stemmer = nltk.stem.PorterStemmer()

    stop_words = set(stopwords.words('english'))
    manual_stop = {'usajobs','announcement','job'
                ,'unit','keyword','search','official'
                ,'open','application','submit','button','must'}
    stop_words = stop_words.union(manual_stop)

    remove_stopwords = [x for x in tokens if not x.lower() in stop_words]
    remove_punct=[word.lower() for word in remove_stopwords if word.isalpha()]

    test_stems = [stemmer.stem(x) for x in remove_punct]

    token_list.append(test_stems)

    bigram_list.append(list(ngrams(test_stems, 2)))
    
    with open(f'./jd-tokens/tokens - {filename}.txt','w') as f:
        for line in test_stems:
            f.write(f"{line}\n")

    with open(f'./jd-bigrams/bigrams - {filename}.txt','w') as f:
        for line in Counter(ngrams(test_stems, 2)):
            f.write(f"{line}\n")

In [147]:
#Add tokens and bigrams to dataframe
data['tokens'] = token_list
data['bigrams'] = bigram_list
data.head()

,job_title,org_name,salary_min,salary_max,pay_type,peer_specialist_flag,require_certification_before_hire,require_certification_after_hire,college_required,url,do_not_overwrite_flag,tokens,bigrams
0,Peer Recovery Specialist,Mile High Behavioral Healthcare,17.29,21.29,Hourly,1,NaN,NaN,NaN,https://www.simplyhired.com/job/FMWclnhxTd_xau...,NaN,"[attent, requir, cloudflar, pleas, enabl, cook...","[(attent, requir), (requir, cloudflar), (cloud..."
1,Peer Support Specialist,ROCKY MOUNTAIN HUMAN SERVICES,20,22,Hourly,1,1.0,NaN,NaN,https://www.linkedin.com/jobs/view/peer-suppor...,NaN,"[rocki, mountain, human, servic, hire, peer, s...","[(rocki, mountain), (mountain, human), (human,..."
2,Peer Specialist,WellPower,25,25,Hourly,1,NaN,NaN,NaN,https://www.linkedin.com/jobs/view/peer-specia...,NaN,"[wellpow, hire, peer, specialist, solut, cente...","[(wellpow, hire), (hire, peer), (peer, special..."
3,Peer Specialist Case Manager,Jefferson Center for Mental Health,"39,100","49,800",Annual,1,NaN,NaN,1.0,https://www.linkedin.com/jobs/view/peer-specia...,NaN,"[jefferson, center, mental, health, hire, peer...","[(jefferson, center), (center, mental), (menta..."
4,Peer Support Specialist,University of Colorado,NaN,NaN,NaN,1,NaN,1.0,NaN,https://www.linkedin.com/jobs/view/peer-suppor...,NaN,"[univers, colorado, hire, peer, support, speci...","[(univers, colorado), (colorado, hire), (hire,..."


In [152]:
#If you want to subset out the job you're comparing to, can do here
director_ds = data[data.job_title=='Director of Data Science'].reset_index()
sample_jobs = data[data.job_title!='Director of Data Science'].reset_index()
all_jobs = data.copy()
director_ds

,index,job_title,org_name,salary_min,salary_max,pay_type,peer_specialist_flag,require_certification_before_hire,require_certification_after_hire,college_required,url,do_not_overwrite_flag,tokens,bigrams


In [153]:
#Extract bigrams of interest
bigram_skill_search = {
    "data_science":('data', 'scienc'),
    "data_engineering":('data', 'engin'),
    "database_design":('databas', 'design'),
    "data_analysis":('data', 'analysi'),
    "collaborate_stakeholders":('collabor', 'stakehold'),
    "data_quality":('data', 'qualiti'),
    "strong_communication":('strong', 'commun'),
    "communication_skills":('commun', 'skill'),
    "data_pipelines":('data', 'pipelin'),
    "unstructured_data":('unstructur', 'data'),
    "data_lake":('data', 'lake'),
    "data_model":('data', 'model'),
    "data_scientist":('data', 'scientist'),
    "data_requirements":('data', 'requir'),
    "team_management":('team', 'manag'),
    "data_manipulation":('data', 'manipul'),
    "data_warehouse":('data', 'wareh'),
    "machine_learning":('machin', 'learn')
}

bigram_requirements = {
    "bachelor_degree":('bachelor', 'degre'),
    "masters_degree":('master', 'doctor')
}

monogram_skill_search = ['sql','python','tableau','r']

In [154]:
#Create bool fields where bigram is detected
for i in bigram_skill_search:
    temp_list = []
    for j in range(0,all_jobs.shape[0]):
        if bigram_skill_search[i] in all_jobs.bigrams[j]:
            temp_list.append(1)
        else:
            temp_list.append(0)
    all_jobs[i] = temp_list

for i in monogram_skill_search:
    temp_list = []
    for j in range(0,all_jobs.shape[0]):
        if i in all_jobs.tokens[j]:
            temp_list.append(1)
        else:
            temp_list.append(0)
    all_jobs[i] = temp_list

all_jobs.head(10)

,job_title,org_name,salary_min,salary_max,pay_type,peer_specialist_flag,require_certification_before_hire,require_certification_after_hire,college_required,url,...,data_scientist,data_requirements,team_management,data_manipulation,data_warehouse,machine_learning,sql,python,tableau,r
0,Peer Recovery Specialist,Mile High Behavioral Healthcare,17.29,21.29,Hourly,1,NaN,NaN,NaN,https://www.simplyhired.com/job/FMWclnhxTd_xau...,...,0,0,0,0,0,0,1,0,0,0
1,Peer Support Specialist,ROCKY MOUNTAIN HUMAN SERVICES,20,22,Hourly,1,1.0,NaN,NaN,https://www.linkedin.com/jobs/view/peer-suppor...,...,0,0,0,0,0,0,0,0,0,0
2,Peer Specialist,WellPower,25,25,Hourly,1,NaN,NaN,NaN,https://www.linkedin.com/jobs/view/peer-specia...,...,0,0,0,0,0,0,0,0,0,0
3,Peer Specialist Case Manager,Jefferson Center for Mental Health,"39,100","49,800",Annual,1,NaN,NaN,1.0,https://www.linkedin.com/jobs/view/peer-specia...,...,0,0,0,0,0,0,0,0,0,0
4,Peer Support Specialist,University of Colorado,NaN,NaN,NaN,1,NaN,1.0,NaN,https://www.linkedin.com/jobs/view/peer-suppor...,...,0,0,0,0,0,0,0,0,0,0
5,"Peer Specialist, Ascent",Community Reach Center,24.04,24.04,Hourly,1,NaN,NaN,NaN,https://www.simplyhired.com/job/dZPDKGYP7bqv7M...,...,0,0,0,0,0,0,1,0,0,0
6,"Peer Specialist I, ACT - Housing First",Colorado Coalition for the Homeless,21.1,25.71,Hourly,1,NaN,NaN,NaN,https://www.linkedin.com/jobs/view/peer-specia...,...,0,0,0,0,0,0,0,0,0,0
7,Peer Support Specialist,Mental Health Partners,NaN,NaN,NaN,1,NaN,1.0,NaN,https://www.salary.com/job/mental-health-partn...,...,0,0,0,0,0,0,0,0,0,0
8,Peer Support Apprentice Peer Specialist,Veterans Health Administration,"41,757","82,249",Annual,1,1.0,NaN,NaN,https://www.ziprecruiter.com/c/Veterans-Health...,...,0,0,0,0,0,0,0,0,0,0
9,PT Residential Peer Specialist,Empowerment Program,17.29,17.29,Hourly,1,NaN,NaN,NaN,https://www.simplyhired.com/job/54ZoW3mR3wU5o-...,...,0,0,0,0,0,0,1,0,0,0


In [150]:
#Create bool fields to categorize job level type
import numpy as np
all_jobs['JobType'] = np.where(
     all_jobs['director_flag']==1,
    'Director', 
     np.where(all_jobs['analyst_flag']==1, 'Data Analyst', 
            np.where(all_jobs['scientist_flag']==1, 'Data Scientist', 
               np.where(all_jobs['manager_flag']==1, 'Manager',
                  np.where(all_jobs['engineer_flag']==1, 'Data Engineer', 
                     'Unknown'
                  )
               )
            )
      )
)

In [120]:
#Output to csv
all_jobs[['job_title', 'org_name', 'salary_min', 'salary_max',
       'director_flag', 'analyst_flag', 'scientist_flag', 'manager_flag',
       'engineer_flag', 'url', 'data_science',
       'data_engineering', 'database_design', 'data_analysis',
       'collaborate_stakeholders', 'data_quality', 'strong_communication',
       'communication_skills', 'data_pipelines', 'unstructured_data',
       'data_lake', 'data_model', 'data_scientist', 'data_requirements',
       'team_management', 'data_manipulation', 'data_warehouse',
       'machine_learning', 'sql', 'python', 'tableau','JobType']].to_csv('simplified_output.csv')

In [155]:
#Output to csv
all_jobs.to_csv('full_output.csv')

In [122]:
#Cluser together bigram skill scoring
perc_similar = all_jobs[['job_title', 'org_name', 'salary_min', 'salary_max',
       'director_flag', 'analyst_flag', 'scientist_flag', 'manager_flag',
       'engineer_flag', 'data_science',
       'data_engineering', 'database_design', 'data_analysis',
       'collaborate_stakeholders', 'data_quality', 'strong_communication',
       'communication_skills', 'data_pipelines', 'unstructured_data',
       'data_lake', 'data_model', 'data_scientist', 'data_requirements',
       'team_management', 'data_manipulation', 'data_warehouse',
       'machine_learning', 'sql', 'python', 'tableau','r']].copy()

In [123]:
#Calculate how many bigram skills should be available
num_skills = len(bigram_skill_search)+len(monogram_skill_search)

In [124]:
#Generate similarity vectors based on number of bigram values found
perc_similar['similarity_count']=perc_similar.iloc[:, -num_skills-1:].sum(axis=1)
perc_similar['similarity_perc']=perc_similar['similarity_count']/num_skills

In [125]:
#Print out top matching jobs
perc_similar[['job_title','similarity_perc']].sort_values(by='similarity_perc',ascending=False).head(10)

,job_title,similarity_perc
27,Director of Data Science,1.000000
36,Data Engineering Specialist,0.590909
35,Senior Data Engineer,0.590909
59,Data Scientist,0.500000
93,Data Scientist,0.500000
78,SENIOR DATA SCIENTIST,0.500000
105,Data Scientist,0.454545
10,Data Scientist,0.454545
14,Senior Data Scientist,0.454545
37,Bloomberg Data Engineer,0.409091


In [127]:
#Print out jobs with no matching values
perc_similar[['job_title','org_name','similarity_perc']][perc_similar.similarity_perc == 0].shape

(35, 3)

In [130]:
domains = [
    ("data_science", "Data Science"),
    ("data_engineering", "Data Engineering"),
    ("database_design", "Database Design"),
    ("data_analysis", "Data Analysis"),
    ("collaborate_stakeholders", "Collaboration with Stakeholders"),
    ("data_quality", "Data Quality"),
    ("strong_communication", "Strong Communication"),
    ("communication_skills", "Communication Skills"),
    ("data_pipelines", "Data Pipelines"),
    ("unstructured_data", "Unstructured Data"),
    ("data_lake", "Data Lake"),
    ("data_model", "Data Modeling"),
    ("data_scientist", "Data Scientist"),
    ("data_requirements", "Data Requirements"),
    ("team_management", "Team Management"),
    ("data_manipulation", "Data Manipulation"),
    ("data_warehouse", "Data Warehousing"),
    ("machine_learning", "Machine Learning"),
]

#Find top matching jobs
options = ["Data Science",
           "Machine Learning"]

[x[0] for x in domains if x[1] in options]

['data_science', 'machine_learning']